## My Black Jack
This is a version of the game of BlackJack that I developed durong the Python Bootcamp course.

In [1]:
'''
Setting up modules and "immutables"
The values dict is specific to the game of Black Jack (10 and figures equal 10 and Ace can be 1 or 11)
'''
# We'll use this later
import random 

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':(11,1)}

In [2]:
'''
Reusing classes from the warm-up part of this project
Classes are Card, Deck and Player
'''
class Card:
    '''
    Card Class from outside variables: suit and rank
    '''
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return self.rank + ' of ' + self.suit

class Deck:
    '''
    Deck class using the Card class to populate its attribute all_cards
    '''
    def __init__(self):
        # Note this only happens once upon creation of a new Deck
        self.all_cards = [] 
        for suit in suits:
            for rank in ranks:
                # This assumes the Card class has already been defined!
                self.all_cards.append(Card(suit,rank))
                
    def shuffle(self):
        # Note this doesn't return anything
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        # Note we remove one card from the list of all_cards
        return self.all_cards.pop()

    def __str__(self):
        return f'Deck size: {len(self.all_cards)} cards.'

class Player:
    
    def __init__(self,name):
        self.name = name
        # A new player has no cards
        self.all_cards = []
        
    def remove_one(self):
        # Note we remove one card from the list of all_cards
        # We state 0 to remove from the "top" of the deck
        # We'll imagine index -1 as the bottom of the deck
        return self.all_cards.pop(0)
    
    def add_cards(self,new_cards):
        if type(new_cards) == type([]):
            self.all_cards.extend(new_cards)
        else:
            self.all_cards.append(new_cards)
    
    def print_cards(self):
        print(f"{self.name} has:")
        for card in self.all_cards:
            print(card)
    
    def __str__(self):
        return f'Player {self.name} has {len(self.all_cards)} cards.'

In [3]:
suit = suits[0]
rank = ranks[12]
ace_hearts = Card(suit,rank)
print(ace_hearts)
print(ace_hearts.value)

Ace of Hearts
(11, 1)


## Specific Black Jack players
In the next section I'll build a class that inherits from Player but has a bit more attributes and functions so that it can be suitable for the game of Black Jack. Namely, in black jack instances of player can be either player or dealer.

In [4]:
class BlackjackPlayer(Player):
    def __init__(self, name):
        Player.__init__(self, name)
        self.cards_sum = 0
        
        self.balance = 0
        self.bet = 0

    def empty_hand(self):
        self.all_cards.clear()

    def sum_of_cards(self):
        self.cards_sum = 0
        aces = []
        # Sum all non-Ace card values
        for card in self.all_cards:
            if card.rank == 'Ace':
                aces.append(card)
            else:
                self.cards_sum += card.value
        # Now handle Aces — count them and add smartly
        for card in aces:
        # Try to count Ace as 11 if it doesn't bust
            if self.cards_sum + card.value[0] <= 21:
                self.cards_sum += card.value[0]
            else:
                self.cards_sum += card.value[1]
        print(f"{self.name} has {self.cards_sum}.\n")

    def bust_check(self):
        if self.cards_sum > 21:
            return True

    def blackjack_check(self):
        if len(self.all_cards) ==2 and self.cards_sum == 21:
            return True

    def deposit(self, amount):
        self.balance += amount
        print(f"{self.name} deposited {amount}. Balance is: {self.balance}\n")

    def pay(self, name, amount):
        self.balance-=amount
        print(f"{amount} chips paid to {name}\n")

    def receive(self, name, amount):
        self.balance+=amount
        print(f"{self.name} got paid {amount} chips")

    def set_bet(self, amount):
        if amount > self.balance:
            print("Insufficient balance in your account")
            print("You can't place a bet.\n")
            return False
        else:
            self.balance-=amount
            self.bet=amount
            print(f"{self.name} bets {self.bet}\n")
            return True

    def return_bet(self):
        self.balance += self.bet
        self.bet = 0

    def reset_bet(self):
        self.bet = 0

## Short tests on added attributes

In [5]:
deck0 = Deck()
player1 = BlackjackPlayer('player1')
print(player1)
while len(player1.all_cards) < 2:
    player1.add_cards(deck0.deal_one())
print(player1)
player1.sum_of_cards()

for card in player1.all_cards:
    print(card)
print(player1.cards_sum)

Player player1 has 0 cards.
Player player1 has 2 cards.
player1 has 21.

Ace of Clubs
King of Clubs
21


In [6]:
player1.set_bet(100)
player1.deposit(10000)
player1.set_bet(100)

Insufficient balance in your account
You can't place a bet.

player1 deposited 10000. Balance is: 10000

player1 bets 100



True

In [7]:
print(player1.balance)

9900


## Blackjack logic

In [8]:
# Setting up the game and players
deck = Deck()
deck.shuffle()

# Setting up Dealer
dealer = BlackjackPlayer("The bank")
# Amount for the Bank
dealer.deposit(10000)

print("---------------------------------")
print("Welcome to the game of Blackjack!")
print(f"Today {dealer.name}'s balance is {dealer.balance}")
print("---------------------------------\n")

# Setting up player
while True:
    name = input("Type in your name: ").strip()
    if name:
        break
    print("Name cannot be empty. Please try again.")
player = BlackjackPlayer(name)
deposit = int(input('''Please make a deposit to your chip account: '''))
player.deposit(deposit)

# Some functions needed during the game
def check_choice():
    while True:
        try:
            choice = input("Type 'hit' or 'stay': ").strip().lower()
            while choice not in ('hit', 'stay'):
                print("Not the right keywords. Try again.")
                choice = input("Type 'hit' or 'stay': ").strip().lower()
            return choice
        except Exception as e:
            print("There was an error:", e)
            continue

def deal_cards():
    while len(dealer.all_cards) <2:
        player.add_cards(deck.deal_one())
        dealer.add_cards(deck.deal_one())

def evaluate(player_sum, dealer_sum):
    if player_sum > dealer_sum:
        return "player"
    elif player_sum == dealer_sum:
        return "draw"
    else:
        return "dealer"

# Let's start the game
print("Let's play!\n")

game_on = True

while game_on and player.balance > 0:

    round = True
    while round:
        # Bet first: ask player how much and set his bet
        while True:
            try:
                bet = int(input("Place your bet: "))
                if bet <= 0:
                    print("Bet must be a positive number.")
                    continue
                elif bet%10 != 0:
                    print("Bet must be in tens.")
                    continue
                if (player.set_bet(bet)):
                    break
                else:
                    continue
            except ValueError:
                print("Invalid input! Please enter a number.")

        # Deal 2 cards each
        deal_cards()

        # Let's see our cards
        player.print_cards()
        player.sum_of_cards()

        # Let's see 1 of the dealer's cards
        print("Dealer has:")
        print(f"{dealer.all_cards[0]}\n")

        # Check for blackjacks
        if player.blackjack_check():
            print(f"{player.name} has BlackJack!")
            if dealer.blackjack_check():
                print(f"{dealer.name} also has BlackJack!")
                print("It's a draw")
                player.return_bet(player.bet)
                print("\n")
                round = False
                break
            else:
                dealer.pay(player.name, (int(1.5*player.bet)))
                player.receive(dealer.name,(int(1.5*player.bet)))
                player.return_bet()
                print("\n")
                round = False
                break
        # Ask for hit or stay
        while player.cards_sum < 21:
            # Hit or stay
            print("What do you want to do?")
            choice = check_choice()
            if choice == 'hit':
                player.add_cards(deck.deal_one())
                player.print_cards()
                player.sum_of_cards()
                if player.bust_check():
                    print("You bust!")
                    dealer.receive(player.name,player.bet)
                    player.reset_bet()
                    round = False
                    break
            else:
                # Stay
                print("\n")
                break
                
        # Getting straight to next round after busting
        if player.cards_sum > 21:
            continue
        
        # Dealer's turn to maybe draw morecards
        dealer.print_cards()
        dealer.sum_of_cards()
        # Dealer has blackjack
        if dealer.blackjack_check():
            print(f"{dealer.name} has BlackJack!")
            print("You lost.")
            dealer.receive(player.name,player.bet)
            player.reset_bet()
            round = False
            break

        while dealer.cards_sum < 17:
            dealer.add_cards(deck.deal_one())
            dealer.print_cards()
            dealer.sum_of_cards()
            
        if dealer.bust_check():
            print(f"{dealer.name} busts!")
            dealer.pay(player.name, player.bet)
            player.receive(dealer.name, player.bet)
            player.return_bet()
            round = False
            break

        # Evaluate winner
        winner = evaluate(player.cards_sum, dealer.cards_sum)
        print("Winner is: "+winner)
        # Pay bets
        if winner == "player":
            dealer.pay(player.name, player.bet)
            player.receive(dealer.name, player.bet)
            player.return_bet()
        elif winner == "draw":
            player.return_bet()
        else:
            dealer.receive(player.name,player.bet)
            player.reset_bet()
        round = False

    while not round:
        print(f'''End of round:
        {player.name} has {player.balance} chips.\n''')
        print("---------------------------------\n")

        if player.balance < 10:
            game_on = False
            print(f"Your balance is at {player.balance}. You can't play anymore. Goodbye!")
            break

        # Checking there's enough cards in the Deck before next round
        if len(deck.all_cards) < 17:
            deck = Deck()
            deck.shuffle()
            print(f"Deck renewed: {deck}")
        
        rnd_choice = str(input("Wanna play another round? Type 'y' for yes, 'n' for no"))
        if rnd_choice == 'y':
            print("---------------------------------")
            print("New round:\n")
            player.empty_hand()
            dealer.empty_hand()
            round = True
        elif rnd_choice == 'n':
            game_on = False
            round = False
            break

The bank deposited 10000. Balance is: 10000

---------------------------------
Welcome to the game of Blackjack!
Today The bank's balance is 10000
---------------------------------



Type in your name:  Francois
Please make a deposit to your chip account:  1000


Francois deposited 1000. Balance is: 1000

Let's play!



Place your bet:  100


Francois bets 100

Francois has:
Jack of Spades
Five of Clubs
Francois has 15.

Dealer has:
Jack of Diamonds

What do you want to do?


Type 'hit' or 'stay':  stay




The bank has:
Jack of Diamonds
Queen of Hearts
The bank has 20.

Winner is: dealer
The bank got paid 100 chips
End of round:
        Francois has 900 chips.

---------------------------------



Wanna play another round? Type 'y' for yes, 'n' for no n
